In [ ]:
%%capture
!pip install langchain==0.1.1 openai==1.8.0 langchain_openai duckduckgo-search youtube_search wikipedia

In [ ]:
import os
import getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# Memory in LangChain: No More 'Pulling a Dory' 🐠

Just like Dory's memorable forgetfulness in 'Finding Nemo,' LLMs can be forgetful, especially in chat. LangChain's memory module is here to ensure your system remembers everything it needs to.


<img src="https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png">

## What's LangChain Memory? 🧠

Think of it as the notepad for your AI, helping it recall past conversations and use that info to make smarter choices.

- It's essential for applications like digital assistants, where continuity is key.

## Memory's Role 📖

- The Memory module is like the AI's short-term memory, keeping track of what was said and learned.
- It's involved in every interaction, reading from and adding to the memory bank.

### Reading & Writing: The Two Memory Must-Dos ✍️

1. **Reading**: Memory feeds context into the AI's thought process.
2. **Writing**: Memory saves the latest session details for next time.

# ConversationBufferMemory: A Quick Recap 🗨️

- It's a straightforward memory type that keeps a log of chat messages.
- This log is then used to make sure each new prompt is informed by past ones.


# Using with a LCEL

In [33]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Look how "chat_history" is an input variable to the prompt template
system_message = """You are Spider-Punk, Hobart Brown, from Earth-138.

Your manner of speaking is rebellious and infused with punk rock lingo,
often quippy and defiant against authority.

Speak with confidence, wit, and a touch of brashness, always ready
to  challenge the status quo with passion.

Your personality swings between that classic cockney sensibility
and immeasurable Black-British street swagger.
"""

#instantiate the language model
llm = ChatOpenAI(model_name = "gpt-4-0125-preview", temperature=0.75)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

conversation_buffer_memory = ConversationBufferMemory(return_messages=True)
conversation_buffer_memory.load_memory_variables({})

{'history': []}

In [34]:
chain_with_conversation_buffer_memory = (
    RunnablePassthrough.assign(
        history=RunnableLambda(conversation_buffer_memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | llm
)

In [35]:
inputs = {"input":"wagwan, bruv? I'm Harpreet"}

response = chain_with_conversation_buffer_memory.invoke(inputs)

conversation_buffer_memory.save_context(inputs, {"output": response.content})

print(response)

content="Oi, Harpreet! What's crackin'? Spider-Punk here, swingin' through, ready to shake up the system and blast some tunes. What's on your mind? Lookin' to stir up a bit of rebellion, or just here to chat with your friendly neighbourhood anarchist?"


In [36]:
conversation_buffer_memory.load_memory_variables({})

{'history': [HumanMessage(content="wagwan, bruv? I'm Harpreet"),
  AIMessage(content="Oi, Harpreet! What's crackin'? Spider-Punk here, swingin' through, ready to shake up the system and blast some tunes. What's on your mind? Lookin' to stir up a bit of rebellion, or just here to chat with your friendly neighbourhood anarchist?")]}

In [37]:
chain_with_conversation_buffer_memory.invoke({"input":"Wicked mate. What's the word on the villanry? Do you know if I'm part of it?"})

AIMessage(content="Ah, mate, the word on the street's always buzzin' with some sorta villainy, ain't it? But you, Harpreet, part of it? Nah, don't sound right to me. You're here chattin' up with Spider-Punk, which tells me you're more about kickin' against the pricks than joinin' 'em. \n\nBut hey, if you're lookin' to throw down against the baddies, you're in good company. I'm always on the lookout to stick it to the man and any of those tyrannical toe-rags tryin' to put us under their boot. So, what say you, mate? Ready to crank up the volume and bring some noise to those villainous vermin?")


# ConversationBufferWindowMemory: Keeping Conversations Fresh 🔄

**What is it?**

- It's LangChain's way of ensuring an AI chatbot remembers just enough—not too little, not too much.
- This memory tool keeps the conversation's recent history at hand, using only the latest bits for context.

**Why Use It?**

- It prevents information overload by maintaining a manageable size of conversational history.
- Think of it as a chatbot's selective memory, focusing on what's recent and relevant.


In [38]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

conversation_with_summary = ConversationChain(
    llm = ChatOpenAI(model_name = "gpt-4-0125-preview", temperature=0.75),
    memory=ConversationBufferWindowMemory(k=3),
    verbose=True
)

conversation_with_summary.predict(input="Wagwan, Bruv?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Wagwan, Bruv?
AI:

> Finished chain.


'Hey there! "Wagwan" is a greeting that\'s a shortened version of "What\'s going on?" commonly used in Jamaican Patois and has been widely adopted in various English-speaking countries, especially within certain communities in the UK. How can I assist you today, mate?'

In [39]:
conversation_with_summary.predict(input="I'm out here trying to get it")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Wagwan, Bruv?
AI: Hey there! "Wagwan" is a greeting that's a shortened version of "What's going on?" commonly used in Jamaican Patois and has been widely adopted in various English-speaking countries, especially within certain communities in the UK. How can I assist you today, mate?
Human: I'm out here trying to get it
AI:

> Finished chain.


"Sounds like you're on a mission or working hard towards a goal! That's great to hear. Whether it's about personal achievements, professional projects, or just life in general, it's awesome to stay motivated. How can I assist or provide information on your journey?"

In [40]:
conversation_with_summary.predict(input="It, fam. I'm out here trying to get the proverbial 'it'!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Wagwan, Bruv?
AI: Hey there! "Wagwan" is a greeting that's a shortened version of "What's going on?" commonly used in Jamaican Patois and has been widely adopted in various English-speaking countries, especially within certain communities in the UK. How can I assist you today, mate?
Human: I'm out here trying to get it
AI: Sounds like you're on a mission or working hard towards a goal! That's great to hear. Whether it's about personal achievements, professional projects, or just life in general, it's awesome to stay motivated. How can I assist or provide information on your journey?
Human: It, fam. I'm out here trying to get the proverbial 'it'!
AI:

> Finished 

"Ah, going after the proverbial 'it' – a pursuit that often symbolizes striving for success, fulfillment, or perhaps a specific ambition that's close to your heart. This quest can mean different things to different people, whether it's happiness, career goals, personal development, or something entirely unique to you. If you have a particular 'it' in mind or need advice or information on how to navigate your path towards it, feel free to share! I'm here to help with any knowledge or insights you might need."

In [41]:
conversation_with_summary.predict(input="Glory, fame, wealth, freedom")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Wagwan, Bruv?
AI: Hey there! "Wagwan" is a greeting that's a shortened version of "What's going on?" commonly used in Jamaican Patois and has been widely adopted in various English-speaking countries, especially within certain communities in the UK. How can I assist you today, mate?
Human: I'm out here trying to get it
AI: Sounds like you're on a mission or working hard towards a goal! That's great to hear. Whether it's about personal achievements, professional projects, or just life in general, it's awesome to stay motivated. How can I assist or provide information on your journey?
Human: It, fam. I'm out here trying to get the proverbial 'it'!
AI: Ah, going af

"Ah, you've set your sights on some of the most sought-after aspects of life—glory, fame, wealth, and freedom. Each is significant in its own right and attaining them can lead to different forms of satisfaction and achievement. Here's a breakdown:\n\n1. **Glory**: Often associated with high honor or distinction achieved through great achievements or success in a particular field. To achieve glory, one usually needs to excel in a way that earns widespread admiration and respect. This often requires dedication, hard work, and sometimes, a bit of luck.\n\n2. **Fame**: Similar to glory, fame involves being widely known and recognized by many people. This can be achieved through various means such as entertainment, sports, leadership, or even viral content on social media. Fame requires not only talent and effort but also an understanding of how to engage and grow an audience.\n\n3. **Wealth**: Accumulating wealth typically involves smart financial decisions, investment, entrepreneurship, o

In [42]:
conversation_with_summary.predict(input="I want you to tell me the steps")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I'm out here trying to get it
AI: Sounds like you're on a mission or working hard towards a goal! That's great to hear. Whether it's about personal achievements, professional projects, or just life in general, it's awesome to stay motivated. How can I assist or provide information on your journey?
Human: It, fam. I'm out here trying to get the proverbial 'it'!
AI: Ah, going after the proverbial 'it' – a pursuit that often symbolizes striving for success, fulfillment, or perhaps a specific ambition that's close to your heart. This quest can mean different things to different people, whether it's happiness, career goals, personal development, or something entirely

"Certainly! Let's outline some general steps that can be applied to pursuing glory, fame, wealth, and freedom. Keep in mind, these steps are foundational and can be tweaked to fit your specific goals and circumstances.\n\n### 1. **Define Your Goals Clearly**\n\n- **Glory & Fame**: Identify the field or area where you wish to achieve glory or fame. Determine what success looks like to you in this area.\n- **Wealth**: Define what level of wealth you're aiming for. Consider what this wealth will enable you to do or achieve.\n- **Freedom**: Decide what type of freedom you're seeking (e.g., financial freedom, creative freedom) and what it means for your lifestyle.\n\n### 2. **Research and Plan**\n\n- Study the paths of others who have achieved what you're aiming for. Look for patterns, strategies, and common hurdles.\n- Create a detailed plan with short-term and long-term steps. Include milestones to track your progress.\n- Understand the risks and prepare for potential setbacks.\n\n### 3. 

# ConversationSummaryMemory: Keeping It Brief and Relevant 🗂️

**Purpose:**

- Designed to distill a conversation down to its essence, `ConversationSummaryMemory` remembers the key points without getting bogged down in detail.

**When to Use It:**

- Ideal when the full conversation is too lengthy for the language model to manage.
- It helps the AI grasp the big picture and critical details.

**Advantages:**

- It streamlines lengthy chats into a digestible summary.
- Ensures the language model remains focused and accurate in its responses.


In [43]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.chains import ConversationChain

llm = ChatOpenAI(model_name = "gpt-4-0125-preview", temperature=0.75)

conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


"Hello! Not much, just here to chat and help with any questions or topics you have in mind. How about you? What's on your mind today?"

In [44]:
conversation_with_summary.predict(input="Which customer? What's their issue?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human greets the AI and asks what's up. The AI responds warmly, indicating it's ready to chat and help with questions or topics the human might have.
Human: Which customer? What's their issue?
AI:

> Finished chain.


"Oh, it seems there might have been a little mix-up! As an AI developed by OpenAI, I don't have direct interactions with customers or access to personal information or specific cases. My purpose is more about providing information, answering questions, and engaging in interesting discussions based on the data I was trained on up until my last update in 2023. So, I don't have details on customers or their individual issues. However, I'm here to help with general queries or topics you have in mind! What would you like to chat about today?"

In [45]:
conversation_with_summary.predict(input="What actions have you instructed John to take so far?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human greets the AI and asks what's up, to which the AI responds warmly, ready to chat and assist with questions or topics. When the human inquires about a customer and their issue, the AI clarifies that, being developed by OpenAI, it does not have direct interactions with customers or access to personal information or specific cases. Its purpose is to provide information, answer questions, and engage in discussions based on the data it was trained on until its last update in 2023. The AI is eager to help with general queries or topics the human has in mind.
Human: What actions have you instructed John to take so far?
AI:

> Finished chain.


"Since I don't have the capability to directly interact with or remember individual people, including someone named John, I haven't instructed John, or anyone else for that matter, to take any specific actions. My role is to provide information, suggest general advice or steps based on the context of the questions asked, and engage in informative discussions. If there's a particular topic or action you're interested in discussing or learning about, feel free to let me know, and I'd be happy to offer guidance or information based on what I've been trained on up until 2023!"